# Imports

In [1]:
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from scipy.stats import mode
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
plt.style.use('dark_background')

# Loading the Dataframe

In [2]:
pwd

'/home/rafael/code/rafabertolace/OnThePitch/notebooks'

## Merging the Seasons csv files (2019-2020 untill 2021-2022)

In [3]:
print(os.listdir('./../raw_data/Netherlands'))

['Eredivise 2019-2020.csv', 'Eredivise 2020-2021.csv', 'Eredivise 2021-2022.csv']


In [4]:
files = [file for file in os.listdir('./../raw_data/Netherlands')]
data = pd.DataFrame()

for file in files:
    df = pd.read_csv('./../raw_data/Netherlands/'+file)
    data = pd.concat([data, df])

In [5]:
data.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,N1,02/08/2019,19:00,Zwolle,Willem II,1,3,A,1,1,...,2.17,-0.25,2.06,1.84,2.02,1.88,2.07,1.90,2.01,1.85
1,N1,03/08/2019,17:30,FC Emmen,Groningen,0,1,A,0,0,...,1.84,0.00,1.92,1.98,1.93,1.97,2.00,2.02,1.92,1.95
2,N1,03/08/2019,17:30,Vitesse,Ajax,2,2,D,1,1,...,2.87,1.00,2.02,1.77,2.10,1.81,2.14,1.84,2.09,1.79
3,N1,03/08/2019,19:45,Twente,PSV Eindhoven,1,1,D,1,0,...,2.41,1.00,2.09,1.81,2.15,1.77,2.16,1.85,2.08,1.79
4,N1,03/08/2019,19:45,VVV Venlo,Waalwijk,3,1,H,0,1,...,2.26,-0.75,2.05,1.75,2.12,1.79,2.15,1.81,2.11,1.77


In [6]:
for col_name in data.columns: 
    print(col_name)

Div
Date
Time
HomeTeam
AwayTeam
FTHG
FTAG
FTR
HTHG
HTAG
HTR
HS
AS
HST
AST
HF
AF
HC
AC
HY
AY
HR
AR
B365H
B365D
B365A
BWH
BWD
BWA
IWH
IWD
IWA
PSH
PSD
PSA
WHH
WHD
WHA
VCH
VCD
VCA
MaxH
MaxD
MaxA
AvgH
AvgD
AvgA
B365>2.5
B365<2.5
P>2.5
P<2.5
Max>2.5
Max<2.5
Avg>2.5
Avg<2.5
AHh
B365AHH
B365AHA
PAHH
PAHA
MaxAHH
MaxAHA
AvgAHH
AvgAHA
B365CH
B365CD
B365CA
BWCH
BWCD
BWCA
IWCH
IWCD
IWCA
PSCH
PSCD
PSCA
WHCH
WHCD
WHCA
VCCH
VCCD
VCCA
MaxCH
MaxCD
MaxCA
AvgCH
AvgCD
AvgCA
B365C>2.5
B365C<2.5
PC>2.5
PC<2.5
MaxC>2.5
MaxC<2.5
AvgC>2.5
AvgC<2.5
AHCh
B365CAHH
B365CAHA
PCAHH
PCAHA
MaxCAHH
MaxCAHA
AvgCAHH
AvgCAHA


# Features Engineering

### Number of Goals, Over and Under

In [7]:
# total number of goals = goals from the home team + goals from visiting team
data['nb_goals']=data['FTHG']+data['FTAG']

# boolean: true or false regarding whether they were more than 2.5 goals
data['over_2.5_goals']=data['nb_goals']>2.5

# boolean: true or false regarding whether they were less than 2.5 goals
data['under_2.5_goals']=data['nb_goals']<2.5

### Payout

  Payout of betting on over/under 2.5 goals: we get 0 if we lose the bet, we get the Avg if we win the bet (Avg = market average of the odds)


#### Payout Opening

In [8]:
# payout under 2.5 for Average OPENING odds
data['payout_avg_under_2.5'] = data['under_2.5_goals']*data['Avg<2.5']

# payout over 2.5 for Average OPENING odds
data['payout_avg_over_2.5'] = data['over_2.5_goals']*data['Avg>2.5']

#payout UNDER 2.5 for PINACLE specifically
data['payout_under_2.5_pinacle'] = data['under_2.5_goals']*data['P<2.5']

#payout OVER 2.5 for PINACLE specifically
data['payout_over_2.5_pinacle'] = data['over_2.5_goals']*data['P>2.5']

#payout UNDER 2.5 for 365 specifically
data['payout_under_2.5_365'] = data['under_2.5_goals']*data['B365<2.5']

#payout OVER 2.5 for 365 specifically
data['payout_over_2.5_365'] = data['over_2.5_goals']*data['B365>2.5']

#### Payout Closing

In [9]:
# payout under 2.5 for Average CLOSING odds
data['payout_avg_under_closing_2.5'] = data['under_2.5_goals']*data['AvgC<2.5']

# payout over 2.5 for Average CLOSING odds
data['payout_avg_over_closing_2.5'] = data['over_2.5_goals']*data['AvgC>2.5']

#payout UNDER 2.5 for PINACLE closing ddds specifically
data['payout_under_2.5_pinacle_closing'] = data['under_2.5_goals']*data['PC<2.5']

#payout OVER 2.5 for PINACLE closing odds specifically
data['payout_over_2.5_pinacle_closing'] = data['over_2.5_goals']*data['PC>2.5']

#payout UNDER 2.5 for 365 closing odds specifically
data['payout_under_2.5_365_closing'] = data['under_2.5_goals']*data['B365C<2.5']

#payout OVER 2.5 for 365 closing odds specifically
data['payout_over_2.5_365_closing'] = data['over_2.5_goals']*data['B365C>2.5']

### Implied Probability

#### Implied Probability Opening

In [10]:
#Implied Probability UNDER 2.5 goals for for overall market opening odds (Avg) 
data['Implied Probability <2.5 avg']=1/data['Avg<2.5']*100

#Implied Probability OVER 2.5 goals for for overall market opening odds (Avg) 
data['Implied Probability >2.5 avg']=1/data['Avg>2.5']*100

#Implied Probability UNDER 2.5 goals for PINACLE
data['Implied Probability <2.5 pinacle']=1/data['P<2.5']*100

#Implied Probability OVER 2.5 goals for PINACLE
data['Implied Probability >2.5 pinacle']=1/data['P>2.5']*100

#Implied Probability UNDER 2.5 goals for 365
data['Implied Probability <2.5 365']=1/data['B365<2.5']*100

#Implied Probability OVER 2.5 goals for 365
data['Implied Probability >2.5 365']=1/data['B365>2.5']*100

#### Implied Probability Closing

In [11]:
#Implied Probability UNDER 2.5 goals for overall market closing odds (AvgC)
data['Implied Probability <2.5 avg closing']=1/data['AvgC<2.5']*100

#Implied Probability OVER 2.5 goals for overall market closing odds (AvgC)
data['Implied Probability >2.5 avg closing']=1/data['AvgC>2.5']*100

#Implied Probability UNDER 2.5 goals for PINACLE closing odds
data['Implied Probability <2.5 pinacle closing']=1/data['PC<2.5']*100

#Implied Probability OVER 2.5 goals for PINACLE closing odds
data['Implied Probability >2.5 pinacle closing']=1/data['PC>2.5']*100

#Implied Probability UNDER 2.5 goals for 365 closing odds
data['Implied Probability <2.5 365 closing']=1/data['B365C<2.5']*100

#Implied Probability OVER 2.5 goals for 365 closing odds
data['Implied Probability >2.5 365 closing']=1/data['B365C>2.5']*100

### Binning the implied probabilities

#### Binning IP Opening

In [12]:
bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]

#Binning UNDER 2.5 Average Market opening odds
data['binned <2.5 avg'] = pd.cut(data['Implied Probability <2.5 avg'], bins)

#Binning Over 2.5 Average Market opening odds
data['binned >2.5 avg'] = pd.cut(data['Implied Probability >2.5 avg'], bins)

#Binned UNDER 2.5 Pinnacle opening odds
data['binned <2.5 pinacle'] = pd.cut(data['Implied Probability <2.5 pinacle'], bins)

#Binned OVER 2.5 Pinnacle
data['binned >2.5 pinacle'] = pd.cut(data['Implied Probability >2.5 pinacle'], bins)

#Binned UNDER 2.5 bet365 OPENING odds
data['binned <2.5 365'] = pd.cut(data['Implied Probability <2.5 365'], bins)

#Binned OVER 2.5 bet365 OPENING odds
data['binned >2.5 365'] = pd.cut(data['Implied Probability >2.5 365'], bins)

#### Binning IP Closing

In [13]:
bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]

#Binning UNDER 2.5 Average Market closing odds
data['binned <2.5 avg closing'] = pd.cut(data['Implied Probability <2.5 avg closing'], bins)

#Binning OVER 2.5 Average Market closing odds
data['binned >2.5 avg closing'] = pd.cut(data['Implied Probability >2.5 avg closing'], bins)

#Binned UNDER 2.5 Pinnacle closing odds
data['binned <2.5 pinacle closing'] = pd.cut(data['Implied Probability <2.5 pinacle closing'], bins)

#Binned OVER 2.5 Pinnacle CLOSING odds
data['binned >2.5 pinacle closing'] = pd.cut(data['Implied Probability >2.5 pinacle closing'], bins)

#Binned UNDER 2.5 bet365 CLOSING odds
data['binned <2.5 365 closing'] = pd.cut(data['Implied Probability <2.5 365 closing'], bins)

#Binned OVER 2.5 bet365 CLOSING odds
data['binned >2.5 365 closing'] = pd.cut(data['Implied Probability >2.5 365 closing'], bins)

# Final Dataset

### Opening Markets

In [14]:
#Average Market UNDER opening odds
fdf_under_avg = data[['Implied Probability <2.5 avg','under_2.5_goals','binned <2.5 avg','payout_avg_under_2.5']]

#Average Market OVER opening odds
fdf_over_avg = data[['Implied Probability >2.5 avg','over_2.5_goals','binned >2.5 avg','payout_avg_over_2.5']]

#pinnacle UNDER opening odds
fdf_under_pinacle = data[['Implied Probability <2.5 pinacle','under_2.5_goals','binned <2.5 pinacle','payout_under_2.5_pinacle']]

#pinnacle OVER opening odds
fdf_over_pinacle = data[['Implied Probability >2.5 pinacle','over_2.5_goals','binned >2.5 pinacle','payout_over_2.5_pinacle']]

#bet365 UNDER opening odds
fdf_under_365 = data[['Implied Probability <2.5 365','under_2.5_goals','binned <2.5 365','payout_under_2.5_365']]

#bet365 OVER opening odds
fdf_over_365 = data[['Implied Probability >2.5 365','over_2.5_goals','binned >2.5 365','payout_over_2.5_365']]

### Closing Markets

In [15]:
#Average Market UNDER closing odds
fdf_under_avg_closing = data[['Implied Probability <2.5 avg closing','under_2.5_goals','binned <2.5 avg closing','payout_avg_under_closing_2.5']]

#Average Market OVER opening odds
fdf_over_avg_closing = data[['Implied Probability >2.5 avg closing','over_2.5_goals','binned >2.5 avg closing','payout_avg_over_closing_2.5']]

#pinnacle UNDER closing odds
fdf_under_pinacle_closing = data[['Implied Probability <2.5 pinacle closing','under_2.5_goals','binned <2.5 pinacle closing','payout_under_2.5_pinacle_closing']]

#pinnacle OVER closing odds
fdf_over_pinacle_closing = data[['Implied Probability >2.5 pinacle closing','over_2.5_goals','binned >2.5 pinacle closing','payout_over_2.5_pinacle_closing']]

#bet365 UNDER closing odds
fdf_under_365_closing = data[['Implied Probability <2.5 365 closing','under_2.5_goals','binned <2.5 365 closing','payout_under_2.5_365_closing']]

#bet365 OVER closing odds
fdf_over_365_closing = data[['Implied Probability >2.5 365 closing','over_2.5_goals','binned >2.5 365 closing','payout_over_2.5_365_closing']]

# Results per implied prob Average Market

### Under

#### Opening Odds

In [16]:
fdf_under_avg_agg = fdf_under_avg.groupby('binned <2.5 avg')['payout_avg_under_2.5'].agg(['count', 'mean', 'median'])

fdf_under_pinacle_agg = fdf_under_pinacle.groupby('binned <2.5 pinacle')['payout_under_2.5_pinacle'].agg(['count', 'mean', 'median'])

fdf_under_bet365_agg = fdf_under_365.groupby('binned <2.5 365')['payout_under_2.5_365'].agg(['count', 'mean', 'median'])

In [17]:
fdf_under_avg_agg.rename(columns={"count": "count_avg", "mean": "mean_avg", "median": "median_avg"}, inplace=True)
fdf_under_avg_agg.index.names = ['bin<2.5']

fdf_under_pinacle_agg.rename(columns={"count": "count_p", "mean": "mean_p", "median": "median_p"}, inplace=True)
fdf_under_pinacle_agg.index.names = ['bin<2.5']

fdf_under_bet365_agg.rename(columns={"count": "count_b365", "mean": "mean_b365", "median": "median_b365"}, inplace=True)
fdf_under_bet365_agg.index.names = ['bin<2.5']

opening_test_1 = fdf_under_avg_agg.merge(fdf_under_pinacle_agg, on='bin<2.5')
opening_test_2 = opening_test_1.merge(fdf_under_bet365_agg, on='bin<2.5')
opening_test_2

,count_avg,mean_avg,median_avg,count_p,mean_p,median_p,count_b365,mean_b365,median_b365
bin<2.5,,,,,,,,,
"(0, 5]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(5, 10]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(10, 15]",1,0.000000,0.000,1,0.000000,0.00,1,0.000000,0.00
"(15, 20]",2,0.000000,0.000,7,0.000000,0.00,6,0.000000,0.00
"(20, 25]",22,0.590455,0.000,22,0.801364,0.00,30,0.701000,0.00
"(25, 30]",35,0.709143,0.000,38,0.837105,0.00,42,0.907143,0.00
"(30, 35]",61,1.148525,0.000,77,1.109740,0.00,53,1.072264,0.00
"(35, 40]",148,1.079730,0.000,182,1.071813,0.00,132,1.103030,0.00
"(40, 45]",221,0.787873,0.000,200,0.891200,0.00,197,0.831472,0.00


#### Closing Odds

In [18]:
fdf_under_avg_closing_gb_agg = fdf_under_avg_closing.groupby('binned <2.5 avg closing')['payout_avg_under_closing_2.5'].agg(['count', 'mean', 'median', mode])

fdf_under_pinacle_closing_gb_agg = fdf_under_pinacle_closing.groupby('binned <2.5 pinacle closing')['payout_under_2.5_pinacle_closing'].agg(['count', 'mean', 'median', mode])

fdf_under_bet365_closing_gb_agg = fdf_under_365_closing.groupby('binned <2.5 365 closing')['payout_under_2.5_365_closing'].agg(['count', 'mean', 'median', mode])

In [19]:
fdf_under_avg_closing_gb_agg.rename(columns={"count": "count_avg", "mean": "mean_avg", "median": "median_avg", "mode": "mode_avg"}, inplace=True)
fdf_under_avg_closing_gb_agg.index.names = ['bin<2.5']

fdf_under_pinacle_closing_gb_agg.rename(columns={"count": "count_p", "mean": "mean_p", "median": "median_p", "mode": "mode_p"}, inplace=True)
fdf_under_pinacle_closing_gb_agg.index.names = ['bin<2.5']

fdf_under_bet365_closing_gb_agg.rename(columns={"count": "count_b365", "mean": "mean_b365", "median": "median_b365", "mode": "mode_b365"}, inplace=True)
fdf_under_bet365_closing_gb_agg.index.names = ['bin<2.5']

closing_test_1 = fdf_under_avg_closing_gb_agg.merge(fdf_under_pinacle_closing_gb_agg, on='bin<2.5')
closing_test_2 = closing_test_1.merge(fdf_under_bet365_closing_gb_agg, on='bin<2.5')
closing_test_2['bins'] = closing_test_2.index
closing_test_2

,count_avg,mean_avg,median_avg,mode_avg,count_p,mean_p,median_p,mode_p,count_b365,mean_b365,median_b365,mode_b365,bins
bin<2.5,,,,,,,,,,,,,
"(0, 5]",0,NaN,NaN,"([], [])",0,NaN,NaN,"([], [])",0,NaN,NaN,"([], [])","(0, 5]"
"(5, 10]",0,NaN,NaN,"([], [])",0,NaN,NaN,"([], [])",0,NaN,NaN,"([], [])","(5, 10]"
"(10, 15]",1,0.000000,0.00,"([0.0], [1])",1,0.000000,0.000,"([0.0], [1])",1,0.000000,0.00,"([0.0], [1])","(10, 15]"
"(15, 20]",11,0.469091,0.00,"([0.0], [10])",15,0.346000,0.000,"([0.0], [14])",15,0.333333,0.00,"([0.0], [14])","(15, 20]"
"(20, 25]",18,0.733333,0.00,"([0.0], [15])",24,0.592500,0.000,"([0.0], [21])",33,0.637273,0.00,"([0.0], [28])","(20, 25]"
"(25, 30]",44,0.567727,0.00,"([0.0], [37])",49,0.886122,0.000,"([0.0], [37])",50,0.974000,0.00,"([0.0], [36])","(25, 30]"
"(30, 35]",74,0.998919,0.00,"([0.0], [50])",94,1.130957,0.000,"([0.0], [59])",71,1.033239,0.00,"([0.0], [46])","(30, 35]"
"(35, 40]",138,1.156449,0.00,"([0.0], [78])",155,1.046516,0.000,"([0.0], [94])",118,1.106780,0.00,"([0.0], [68])","(35, 40]"
"(40, 45]",200,0.878700,0.00,"([0.0], [125])",181,0.924420,0.000,"([0.0], [110])",174,0.831034,0.00,"([0.0], [112])","(40, 45]"


### Over

#### Opening Odds

In [20]:
fdf_over_avg_agg = fdf_over_avg.groupby('binned >2.5 avg')['payout_avg_over_2.5'].agg(['count', 'mean', 'median'])

fdf_over_pinacle_agg = fdf_over_pinacle.groupby('binned >2.5 pinacle')['payout_over_2.5_pinacle'].agg(['count', 'mean', 'median'])

fdf_over_bet365_agg = fdf_over_365.groupby('binned >2.5 365')['payout_over_2.5_365'].agg(['count', 'mean', 'median'])

In [21]:
fdf_over_avg_agg.rename(columns={"count": "count_avg", "mean": "mean_avg", "median": "median_avg"}, inplace=True)
fdf_over_avg_agg.index.names = ['bin>2.5']

fdf_over_pinacle_agg.rename(columns={"count": "count_p", "mean": "mean_p", "median": "median_p"}, inplace=True)
fdf_over_pinacle_agg.index.names = ['bin>2.5']

fdf_over_bet365_agg.rename(columns={"count": "count_b365", "mean": "mean_b365", "median": "median_b365"}, inplace=True)
fdf_over_bet365_agg.index.names = ['bin>2.5']

over_opening_test_1 = fdf_over_avg_agg.merge(fdf_over_pinacle_agg, on='bin>2.5')
over_opening_test_2 = over_opening_test_1.merge(fdf_over_bet365_agg, on='bin>2.5')
over_opening_test_2

,count_avg,mean_avg,median_avg,count_p,mean_p,median_p,count_b365,mean_b365,median_b365
bin>2.5,,,,,,,,,
"(0, 5]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(5, 10]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(10, 15]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(15, 20]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(20, 25]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(25, 30]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(30, 35]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(35, 40]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(40, 45]",4,1.145000,1.145,8,1.717500,2.255,4,1.162500,1.15


#### Closing Odds

In [22]:
fdf_over_avg_closing_agg = fdf_over_avg_closing.groupby('binned >2.5 avg closing')['payout_avg_over_closing_2.5'].agg(['count', 'mean', 'median'])

fdf_over_pinacle_closing_agg = fdf_over_pinacle_closing.groupby('binned >2.5 pinacle closing')['payout_over_2.5_pinacle_closing'].agg(['count', 'mean', 'median'])

fdf_over_bet365_closing_agg = fdf_over_365_closing.groupby('binned >2.5 365 closing')['payout_over_2.5_365_closing'].agg(['count', 'mean', 'median'])

In [23]:
fdf_over_avg_closing_agg.rename(columns={"count": "count_avg", "mean": "mean_avg", "median": "median_avg"}, inplace=True)
fdf_over_avg_closing_agg.index.names = ['bin>2.5']

fdf_over_pinacle_closing_agg.rename(columns={"count": "count_p", "mean": "mean_p", "median": "median_p"}, inplace=True)
fdf_over_pinacle_closing_agg.index.names = ['bin>2.5']

fdf_over_bet365_closing_agg.rename(columns={"count": "count_b365", "mean": "mean_b365", "median": "median_b365"}, inplace=True)
fdf_over_bet365_closing_agg.index.names = ['bin>2.5']

over_closing_test_1 = fdf_over_avg_closing_agg.merge(fdf_over_pinacle_closing_agg, on='bin>2.5')
over_closing_test_2 = over_closing_test_1.merge(fdf_over_bet365_closing_agg, on='bin>2.5')
over_closing_test_2

,count_avg,mean_avg,median_avg,count_p,mean_p,median_p,count_b365,mean_b365,median_b365
bin>2.5,,,,,,,,,
"(0, 5]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(5, 10]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(10, 15]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(15, 20]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(20, 25]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(25, 30]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(30, 35]",0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
"(35, 40]",0,NaN,NaN,2,1.275000,1.275,0,NaN,NaN
"(40, 45]",14,1.142143,1.115,17,1.220588,2.230,11,1.245455,2.25


# Starting Model

In [24]:
#Pedro's pet! Don't touch it!
"""odd = 2.15
iproba = 1/odd*100
for x in range(0, 20):
    if iproba in closing_test_2.iloc[6]['bins']"""

"odd = 2.15\niproba = 1/odd*100\nfor x in range(0, 20):\n    if iproba in closing_test_2.iloc[6]['bins']"

In [25]:
data_model_over_under = data.copy()

In [26]:
data_model_over_under = data_model_over_under[["Div",
"Date", "Time", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "nb_goals", "Avg>2.5", "Avg<2.5", "B365>2.5", "B365<2.5", "P>2.5", 
"P<2.5", "AvgC>2.5", "AvgC<2.5", "B365C>2.5", "B365C<2.5", "PC>2.5", "PC<2.5", "over_2.5_goals", "under_2.5_goals", 
"payout_avg_under_2.5", "payout_avg_over_2.5", "payout_under_2.5_pinacle", "payout_over_2.5_pinacle", "payout_under_2.5_365", 
"payout_over_2.5_365", "payout_avg_under_closing_2.5", "payout_avg_over_closing_2.5", "payout_under_2.5_pinacle_closing", 
"payout_over_2.5_pinacle_closing", "payout_under_2.5_365_closing", "payout_over_2.5_365_closing", "Implied Probability <2.5 avg", 
"Implied Probability >2.5 avg", "Implied Probability <2.5 pinacle", "Implied Probability >2.5 pinacle", "Implied Probability <2.5 365", 
"Implied Probability >2.5 365", "Implied Probability <2.5 avg closing", "Implied Probability >2.5 avg closing", "Implied Probability <2.5 pinacle closing", 
"Implied Probability >2.5 pinacle closing", "Implied Probability <2.5 365 closing", "Implied Probability >2.5 365 closing", 
"binned <2.5 avg", "binned >2.5 avg", "binned <2.5 pinacle", "binned >2.5 pinacle", "binned <2.5 365", "binned >2.5 365", 
"binned <2.5 avg closing", "binned >2.5 avg closing", "binned <2.5 pinacle closing", "binned >2.5 pinacle closing", 
"binned <2.5 365 closing", "binned >2.5 365 closing"]]

In [27]:
data_model_over_under.dropna(inplace=True)

In [28]:
data_model_over_under

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,nb_goals,Avg>2.5,Avg<2.5,...,binned <2.5 pinacle,binned >2.5 pinacle,binned <2.5 365,binned >2.5 365,binned <2.5 avg closing,binned >2.5 avg closing,binned <2.5 pinacle closing,binned >2.5 pinacle closing,binned <2.5 365 closing,binned >2.5 365 closing
0,N1,02/08/2019,19:00,Zwolle,Willem II,1,3,4,1.63,2.27,...,"(40, 45]","(60, 65]","(45, 50]","(60, 65]","(45, 50]","(55, 60]","(45, 50]","(55, 60]","(45, 50]","(55, 60]"
1,N1,03/08/2019,17:30,FC Emmen,Groningen,0,1,1,1.73,2.10,...,"(45, 50]","(55, 60]","(45, 50]","(55, 60]","(50, 55]","(50, 55]","(50, 55]","(50, 55]","(55, 60]","(45, 50]"
2,N1,03/08/2019,17:30,Vitesse,Ajax,2,2,4,1.46,2.66,...,"(35, 40]","(65, 70]","(35, 40]","(65, 70]","(30, 35]","(70, 75]","(30, 35]","(70, 75]","(30, 35]","(70, 75]"
3,N1,03/08/2019,19:45,Twente,PSV Eindhoven,1,1,2,1.44,2.74,...,"(35, 40]","(65, 70]","(35, 40]","(65, 70]","(40, 45]","(60, 65]","(40, 45]","(60, 65]","(40, 45]","(60, 65]"
4,N1,03/08/2019,19:45,VVV Venlo,Waalwijk,3,1,4,1.64,2.22,...,"(40, 45]","(60, 65]","(45, 50]","(60, 65]","(40, 45]","(60, 65]","(40, 45]","(60, 65]","(40, 45]","(60, 65]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,N1,15/05/2022,13:30,Heracles,Sparta Rotterdam,1,3,4,2.19,1.68,...,"(55, 60]","(40, 45]","(60, 65]","(45, 50]","(60, 65]","(40, 45]","(60, 65]","(40, 45]","(60, 65]","(40, 45]"
302,N1,15/05/2022,13:30,Nijmegen,For Sittard,0,1,1,1.66,2.21,...,"(40, 45]","(55, 60]","(45, 50]","(60, 65]","(40, 45]","(65, 70]","(35, 40]","(65, 70]","(35, 40]","(65, 70]"
303,N1,15/05/2022,13:30,Vitesse,Ajax,2,2,4,1.41,2.86,...,"(30, 35]","(65, 70]","(30, 35]","(70, 75]","(30, 35]","(70, 75]","(30, 35]","(70, 75]","(30, 35]","(70, 75]"
304,N1,15/05/2022,13:30,Willem II,Utrecht,3,0,3,1.56,2.39,...,"(40, 45]","(60, 65]","(40, 45]","(60, 65]","(40, 45]","(65, 70]","(40, 45]","(60, 65]","(40, 45]","(65, 70]"


In [30]:
ohe = OneHotEncoder(sparse=False)
ohe.fit(data_model_over_under[['binned <2.5 pinacle']])
bins_encoded = ohe.transform(data_model_over_under[['binned <2.5 pinacle']])
data_model_over_under["(20, 25]"], data_model_over_under["(25, 30]"], data_model_over_under["(30, 35]"], data_model_over_under["(35, 40]"], data_model_over_under["(40, 45]"], data_model_over_under["(45, 50]"], data_model_over_under["(50, 55]"], data_model_over_under["(55, 60]"], data_model_over_under["(60, 65]"], data_model_over_under["(65, 70]"]  = bins_encoded.T
data_model_over_under.drop(columns='binned <2.5 pinacle', inplace=True)

ValueError: too many values to unpack (expected 10)

In [ ]:
data_model_over_under

In [ ]:
data_model_over_under['Pin_pays_better_under'] = data_model_over_under['PC<2.5'] > data_model_over_under['AvgC<2.5']

In [ ]:
data_model_over_under['%vig_p'] = (1 - (1 / (1/data_model_over_under['PC>2.5'] + 1/data_model_over_under['PC<2.5'])))*100

In [ ]:
data_model_over_under['%vig_avg'] = (1 - (1 / (1/data_model_over_under['AvgC>2.5'] + 1/data_model_over_under['AvgC<2.5'])))*100

In [ ]:
#data_model_over_under['pin_diff_avg'] = data_model_over_under['PC<2.5']/data_model_over_under['AvgC<2.5']

In [ ]:
data_model_over_under['PC<2.5_P_relative_diff'] = data_model_over_under['PC<2.5'] > data_model_over_under['P<2.5']

In [ ]:
test_model = data_model_over_under.drop(columns=['FTHG', 'FTAG', 'nb_goals', 'over_2.5_goals', 'under_2.5_goals', 
                                                'Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'binned <2.5 avg', 
                                                'binned >2.5 avg', 'binned >2.5 pinacle', 'binned <2.5 365', 
                                                 'binned >2.5 365', 'binned <2.5 avg closing', 'binned >2.5 avg closing', 
                                                'binned <2.5 pinacle closing', 'binned >2.5 pinacle closing', 'binned <2.5 365 closing', 
                                                'binned >2.5 365 closing', 
                                                'B365>2.5', 'B365<2.5', 'B365C>2.5', 'B365C<2.5', 'payout_under_2.5_365', 'payout_over_2.5_365', 
                                                'payout_under_2.5_365_closing', 'payout_over_2.5_365_closing', 
                                                'Implied Probability <2.5 365', 'Implied Probability >2.5 365', 
                                                'Implied Probability <2.5 365 closing', 'Implied Probability >2.5 365 closing', 
                                                'payout_avg_under_2.5', 'payout_avg_over_2.5', 'payout_under_2.5_pinacle', 
                                                'payout_avg_over_closing_2.5', 'payout_over_2.5_pinacle_closing', 
                                                'Implied Probability <2.5 avg', 'Implied Probability >2.5 avg', 
                                                'Implied Probability <2.5 pinacle', 'Implied Probability >2.5 pinacle', 
                                                'Avg>2.5', 'P>2.5', 'AvgC>2.5', 'PC>2.5', 'payout_over_2.5_pinacle', 
                                                'Implied Probability >2.5 avg closing', 'Implied Probability >2.5 pinacle closing', 
                                                'payout_avg_under_closing_2.5', 'Avg<2.5', 'P<2.5', 'AvgC<2.5', 'PC<2.5', 
                                                'Implied Probability <2.5 avg closing', 'Implied Probability <2.5 pinacle closing'])

test_model

In [ ]:
test_model.columns

In [ ]:
test_model['payout_under_2.5_pinacle_closing'] = test_model['payout_under_2.5_pinacle_closing'] > 1

In [ ]:
X = test_model.drop(columns='payout_under_2.5_pinacle_closing')
y = test_model['payout_under_2.5_pinacle_closing']

model = LogisticRegression()
model.fit(X, y)
# get importance
importance = model.coef_[0]

# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))

In [ ]:
#Features that decrease our chances
for x in [2, 1, 0, 13, 4, 12, 11]:
    print(f'{X.columns[x]}: {importance[x]}')

In [ ]:
#Features that increase our chances
for x in [6, 8, 7, 5, 9, 10, 3]:
    print(f'{X.columns[x]}: {importance[x]}')